Analyse des données

Préparation des données avec pandas


In [2]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

import pandas as pd
import os
import sys
import pycountry

sys.path.append(os.path.abspath(".."))

pd.set_option("display.float_format", lambda x: f"{x :.2f}")
pd.set_option("display.max_columns", None)
from get_dataframes import GetDataframes

from tools import import_config

from plotting import (
    movies_by_decades,
    movies_by_genres,
    movies_by_duration_boxplot,
    movies_by_country,
    movies_duration_by_decades_boxplot,
    movies_top_x,
)

- On importe la dataframe "title_basics" qui contient les infos pour tout les films de la base de données.
- On enlève tout les films à caractère pornographique de notre liste.
- On nettoie les /N et les transforme en 0.
- On change les types de données des colonnes (float en int, etc...)
- On merge la dataframe "title_basics" avec "tmdb_full" pour obtenir des infos supplémentaires sur chaque film.
- On supprime les colonnes inutiles.
- On supprime les lignes pour lesquelles on a des valeurs manquantes suite au merging.
- On transforme les données dans les colonnes ayant plusieurs valeurs et les on les transforme en listes.
- On importe les dataframes "name_basics", "title_akas" et "title_principals".
- On merge la dataframe "title_akas" avec "tmdb_full".
- On filtre les films ayant eu une adaptaion en FR uniquement.
- On filtre seulement les films qui sont "Released".
- On nettoie "name_basics" comme précédement.
- On merge "name_basics", "title_principals" et on trie les acteurs, directeurs et autres dans un autre dataframe.
- On merge chaque dataframe (acteurs, directeurs, etc...) avec la liste des films qui étaient déjà merged avec tmdb.

In [3]:
import polars as pl
df = pd.read_parquet("../streamlit/datasets/site_web.parquet")
df.titre_genres.unique()

array(['Comédie', 'Action, Thriller, Crime', 'Romance, Comédie, Drame',
       ..., 'Animation, Comédie, Familial, Musique',
       'Drame, Histoire, Familial', 'Comédie, Action, Thriller'],
      dtype=object)

In [7]:
df[df.keywords.str.contains("christmas")]

,budget,titre_genres,tmdb_id,titre_id,overview,popularity,production_countries,date,revenue,runtime,tagline,titre_str,rating_avg,rating_vote,keywords,production_companies_name,actors,actors_ids,director,director_ids,url,image,youtube,titre_clean
151,30000000,"Comédie, Romance",549053,tt8623904,Kate est une jeune femme abonnée aux mauvaises...,31.70,"[GB, US]",2019,121550750,102,Parfois tu dois juste avoir la foi.,Last Christmas,7.24,2166,"london, england, holiday, romantic comedy, sin...","[Universal Pictures, Feigco Entertainment]","Emilia Clarke, Henry Golding, Michelle Yeoh","[1223786, 1807132, 1620]",Paul Feig,[116805],https://www.imdb.com/title/tt8623904,https://image.tmdb.org/t/p/w500/sZDXjZG9Bfuq1V...,https://www.youtube.com/watch?v=tU3orsfQfwg,last christmas
258,65000000,"Drame, Thriller, Mystère",345,tt0120663,Conviés à une soirée donnée par leur ami Victo...,40.99,"[GB, US]",1999,162100000,159,,Eyes Wide Shut,7.47,5639,"new york city, prostitute, based on novel or b...","[Stanley Kubrick Productions, Hobby Films, War...","Tom Cruise, Nicole Kidman","[500, 2227]",Stanley Kubrick,[240],https://www.imdb.com/title/tt0120663,https://image.tmdb.org/t/p/w500/4jVpS9XAHdD5zx...,https://www.youtube.com/watch?v=dQw4w9WgXcQ,eyes wide shut
264,80000000,"Action, Fantastique",364,tt0103776,Abandonné à la naissance à cause de sa difform...,40.68,[US],1992,280000000,126,,Batman : Le Défi,6.92,6044,"double life, corruption, darkness, crime fight...","[Warner Bros. Pictures, Polygram Pictures]","Michael Keaton, Danny DeVito, Michelle Pfeiffer","[2232, 518, 1160]",Tim Burton,[510],https://www.imdb.com/title/tt0103776,https://image.tmdb.org/t/p/w500/aExqFVyVwqKClv...,https://www.youtube.com/watch?v=SU8QKdCk6VY,batman : le défi
337,75000000,"Animation, Familial, Comédie, Fantastique",360920,tt2709692,Terré dans sa cave remplie d’inventions et d’e...,163.44,[US],2018,508575295,105,Il n’est jamais trop tôt pour détester Noël,Le Grinch,6.83,3443,"holiday, surrealism, remake, based on children...","[Illumination, Dr. Seuss Enterprises, Universa...","Benedict Cumberbatch, Rashida Jones, Kenan Tho...","[71580, 80591, 77330]","Scott Mosier, Yarrow Cheney","[20503, 61300]",https://www.imdb.com/title/tt2709692,https://image.tmdb.org/t/p/w500/36a2GG5L0QW7iP...,https://www.youtube.com/watch?v=i2RQ-TgiAvw,le grinch
362,40000000,"Comédie, Romance, Drame",508,tt0314331,"L'amour est partout, imprévisible, inexplicabl...",40.86,"[FR, GB]",2003,247933248,135,19 personnes très différentes qui partagent la...,Love Actually,7.11,6104,"london, england, usa president, rock star, sch...","[Working Title Films, DNA Films, Universal Pic...","Alan Rickman, Bill Nighy, Colin Firth","[4566, 2440, 5472]",Richard Curtis,[7018],https://www.imdb.com/title/tt0314331,https://image.tmdb.org/t/p/w500/rJAML5icaPEkj8...,https://www.youtube.com/watch?v=g5doBFqKKpk,love actually
396,28000000,"Action, Thriller",562,tt0095016,"John McClane, policier new‐yorkais, est venu r...",51.09,[US],1988,140767956,132,40 étages en otage !,Piège de cristal,7.78,10312,"based on novel or book, s.w.a.t., hostage, fbi...","[Gordon Company, Silver Pictures, 20th Century...","Bruce Willis, Alan Rickman, Alexander Godunov","[62, 4566, 7674]",John McTiernan,[1090],https://www.imdb.com/title/tt0095016,https://image.tmdb.org/t/p/w500/vav96OZPkzbZhC...,https://www.youtube.com/watch?v=XDg9AkYRWCc,piège de cristal
511,123000000,"Familial, Comédie, Fantastique",8871,tt0170016,Le Grinch est un croque-mitaine de poils verts...,166.24,"[DE, US]",2000,345823040,104,,Le Grinch,6.74,6755,"new love, based on novel or book, holiday, chr...","[Universal Pictures, Imagine Entertainment, LU...","Jim Carrey, Taylor Momsen, Jeffrey Tambor","[206, 20480, 4175]",Ron Howard,[6159],https://www.imdb.com/title/tt0170016,https://image.tmdb.org/t/p/w500/cmbQRAux3YdOII...,https://www.youtube.com/watch?v=s8tc12mDbOY,le grinch
561,0,"Romance, Comédie",525041,tt8709036,L'heure du mariage de l'année a enfin sonné ! ...,13.56,[US],2

In [3]:
def get_genre_df(df: pd.DataFrame, genres: str):
    sort_by = [
        'popularity', 'rating_avg', 'rating_vote'
    ]
    ascending_ = [False for i in range(3)]
    condi = (
        (
            df["titre_genres"].str.contains(genres) &
            ~df["titre_genres"].str.contains("Animation")
        )
        if genres != "Animation"
        else df["titre_genres"].str.contains(genres)
    )
    return df[condi].sort_values(by=sort_by, ascending=ascending_)

drama_genre = get_genre_df(df, "Drame")
comedy_genre = get_genre_df(df, "Comédie")
horror_genre = get_genre_df(df, "Horreur")
animation_genre = get_genre_df(df, "Animation")
scifi_genre = get_genre_df(df, "Science-Fiction")
thriller_genre = get_genre_df(df, "Crime")
top = 10

In [4]:
config = import_config(add=True)
datas = GetDataframes(config)
link = "movies"
df = datas.get_dataframes(link)

2023-11-22 08:57:10 INFO     TSV files already exist.
2023-11-22 08:57:10 INFO     Parquet loaded ! Importing movies...
2023-11-22 08:57:10 INFO     Dataframe movies ready to use!


- On créée une visualisation pour voir la distribution des notes moyennes de chaque film, le nombre de films par décénnie et le total de votes par décénnie.

In [5]:
def iso_to_country_name(iso_codes):
    country_names = []
    for code in iso_codes:
        country = pycountry.countries.get(alpha_2=code)
        if country:
            country_names.append(country.name)
        else:
            country_names.append("Inconnu")
    return ", ".join(country_names)


df["production_countries"] = df["production_countries"].apply(
    iso_to_country_name
)

In [6]:
df[df["titre_duree"] == df["titre_duree"].max()]

,titre_id,titre_str,titre_date_sortie,titre_duree,titre_genres,rating_avg,rating_votes,original_language,original_title,popularity,production_countries,revenue,spoken_languages,status,region,cuts
44929,tt3854496,Ambiancé,2020,43200,"[Fantasy, Drama]",3.40,116,sv,Ambiancé,1.82,Sweden,0,[sv],Released,FR,>2020


In [7]:
figs = movies_by_decades(df)

for fig in figs:
    fig.show()

- On cherche quel genre de films est le plus répandu dans le dataset filtré

In [8]:
movies_by_genres(df)

- On peut voir que le nombre de films en dessous des années 1960 est très bas et nous décidons de ne pas les garder.
- Les films ayant moins de 5 en note moyenne ne nous intéressent pas non plus.
- Et les films ayant reçu trop peu de votes sont aussi enlevés.

In [9]:
condi = (
    (df["titre_date_sortie"] >= 1960)
    & (df["rating_avg"] >= 6.3)
    & (df["rating_votes"] >= 5004)
)

df = df[condi].reset_index(drop="index")

- Ici nous filtrons les genres qui n'ont que peu d'interet pour le cinéma.

In [10]:
genres_ = ["Music", "Documentary", "Reality-TV", "News"]
df = df[
    df["titre_genres"].apply(lambda x: all(g not in x for g in genres_))
]

In [11]:
movies_by_genres(df)

- On peut désormais ré-afficher la distribution des notes moyennes, le total des films par décénnie et le nombre moyen des votes par décénnies

In [12]:
figs = movies_by_decades(df)

for fig in figs:
    fig.show()

- On cherche des informations sur la durée des films en minutes

In [13]:
movies_by_duration_boxplot(df)

- On constate qu'il y a beaucoup trop de films avec des durées bien trop longues ainsi que des films trop courts qui devraient être considéré plutot comme des court-métrages.
- On enlève donc les films en dessous de 62 minutes et au dessus de 210 minutes (3h30)

In [14]:
condi = ~((df["titre_duree"] < 62) | (df["titre_duree"] > 210))
df = df[condi].reset_index(drop="index")

movies_by_duration_boxplot(df)

- On cherche les 10 premiers pays desquels proviennent les films du dataset.

In [15]:
movies_by_country(df)

- On cherche si il y a une augmentation ou diminution de la durée des films dans le temps.

In [16]:
movies_duration_by_decades_boxplot(df)

In [17]:
# df[((df["titre_duree"] > 170) & (df["titre_date_sortie"] > 2020))]

- On cherche les 10 films avec le plus de votes sans compter les documentaires, concerts et films d'animations.

In [18]:
movies_top_x(df, top=10)

In [19]:
# #  World record of the longest documentary
# # source :https://fr.wikipedia.org/wiki/Ambianc%C3%A9
# longest = actor[actor["titre_duree"] == 43200]
# n = longest[["titre_str", "titre_duree"]].iloc[0]
# heure = n["titre_duree"] // 60
# jours = heure // 24
# print(f"Le film le plus long est {n['titre_str']} d'une durée de {heure} heures soit {jours} jours")
# # All time median for duration
# print(f"All time duration median : {round(actor['titre_duree'].median())} minutes")